In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_table('log.intEnergy')
df.head()

FileNotFoundError: File b'log.intEnergy' does not exist

In [ ]:
fileName = "log.intEnergy"
runNum = 1

search = "Step"
n = 1
f = open(fileName)
df = {}
EOF = False
while True:
    try:
        line = f.readline()
    except:
        break
    if search in line:
        if n == runNum:
            keys = line.split()
            for key in keys:
                df[key] = []
            while True:
                try:
                    line = f.readline()
                except:
                    EOF = True
                    break
                try:
                    values = list(float(x) for x in line.split())
                except:
                    break
                for i in range(len(keys)):
                    df[keys[i]].append(values[i])
        else:
            n += 1
    else:
        pass
    if len(df.keys()) > 0 or EOF:
        break
df = pd.DataFrame(df)
f.close()
print(df)

# Working Log file Reader 

test code for class


In [ ]:
filename = 'log.interfaceTracking'   #takes in filename
thermoLabels = 'Step Atoms Temp Press TotEng'.split()    #take in thermo output Labels/arguements
includeLabelsForNewRun = False

dataLines = []

f = open(filename,'r')
data = False
for line in f.readlines():
    #print(line.split())
    
    if data:
        if len(line.split())!= 0 and line.split()[0].isdigit():
            dataLines.append(line.split())
        else:
            data = False
    
    if line.split() == thermoLabels:
        data = True
        if includeLabelsForNewRun:
            dataLines.append(line.split())
    
f.close()

for i in dataLines:
    print(i)

#print(outputLabels)

In [ ]:
df = pd.DataFrame(dataLines,columns=thermoLabels)
#print(df)

Actual Reader Class

In [ ]:
class LogReader:
    def __init__(self,filename,thermoLabels):
        self.var = "holder"
        self.filename = filename
        self.thermoLabels = thermoLabels.split()
    
    # get lines of thermo output data as list of lines
    def dataExtracter(self,includeLabelsForNewRun=False):
        self.includeLabelsForNewRun = includeLabelsForNewRun

        dataLines = []
        
        f = open(filename,'r')
        isDataLine = False
        for line in f.readlines():
            if isDataLine:
                if len(line.split())!= 0 and line.split()[0].isdigit():
                    dataLines.append(line.split())
                else:
                    isDataLine = False
            
            if line.split() == self.thermoLabels:
                isDataLine = True
                if includeLabelsForNewRun:     #used to seperate runs by thermoLabels
                    dataLines.append(line.split())

        f.close()
        return dataLines

    #User method to get dataframe from log data
    def getDataFrame(self,seperateRunsByThermoLabels=False):
        return pd.DataFrame(self.dataExtracter(seperateRunsByThermoLabels),columns=self.thermoLabels)

In [ ]:
reader = LogReader('log.interfaceTracking','Step Atoms Temp Press TotEng Enthalpy')
df = reader.getDataFrame()
print(reader.getDataFrame())

# Working Grapher

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#collection of methods that analyze dataframe data
class LogAnalyzer:
    def __init__(self,dataframe,timestep=1):
        self.df = dataframe
        self.timestep = timestep
        
    def generalPlot(self,X,Y,xlabel='',ylabel='',title=''):
        plt.plot(X,Y)
        plt.title(title)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        #plt.legend(loc='upper left')
        plt.grid()
        plt.show()
        
    def plotColumn(self,columnLabel):
        Y = self.df[columnLabel].values.astype(float)
        X = np.linspace(0,self.timestep*len(Y),len(Y))
        self.generalPlot(X,Y,'Runtime',columnLabel)

In [ ]:
LA = LogAnalyzer(df)
LA.plotColumn('TotEng')


# Reading position vectors from dump files; Working DumpReader class



#implementation assumes that atom ids are in same order for each run, seems to be the case need to confirm 

In [ ]:

class DumpReader:
    def __init__(self,filename, outputLabels = ''):
        self.filename = filename
        self.outputLabels = ('ITEM: ATOMS '+ outputLabels).split()
    
    def dataExtracter(self):
        dataLines = []         # format : [[rowData1], [rowData2], [rowData3], ... ]
        self.idLabels = ['Timestep']      # format : ['Timestep', 'atomID', 'atomID', ... ]
        rowData = []                      # format : [[timestep], [x,y,z], [x,y,z], ... ]
        
        f = open(self.filename,'r')
        isFirstRun = True
        isDataLine = False          # when initially set, line in next iteration is a data line
        isTimestepLine = False      # when initially set, line in next iteration is a timestep line
        timestep = 0
        
        for line in f.readlines():  
            if line.strip() == 'ITEM: TIMESTEP' or isTimestepLine:
                if isTimestepLine:
                    isTimestepLine = False
                    timestep = int(line.split()[0])
                    rowData.append([timestep])
                    if timestep > 0:                 #assumes simulation starts at 0 timestep
                        isFirstRun = False
                else:
                    isTimestepLine = True
            
            if isDataLine:  
                data = line.split()
                if len(line.split())!= 0 and data[0].isdigit():   
                    if isFirstRun: 
                        self.idLabels.append(data[0])   # append label
                    rowData.append(line.split()[1:])   
                else:
                    isDataLine = False
                    dataLines.append(rowData)
                    rowData = []
            
            if line.split() == self.outputLabels:    
                isDataLine = True                     
                
        f.close()
        return dataLines

    #User method to get dataframe from log data
    def getDataframe(self):
        return pd.DataFrame(self.dataExtracter(),columns=self.idLabels)
    
    def getNdArray(self):
        data = self.dataExtracter()
        for row in range(len(data)):
            #data[row] = np.array(data[row])
            for col in range(len(data[row])):
                data[row][col] = np.array(data[row][col]).astype(float)
        return np.array(data)
'''
note, tried to build dataLines as np.array but since i consistently appending new data to rowData i 
have to make a new copy and reassign (very wasteful) 
instead trying building dataLines then converting elements to np.array by looping, (is there a more efficient
way to do this?)

'''


code for testing/proof of concept 

In [ ]:
#outputLabels = 'id x y z'
DR = DumpReader('atomParameters.data','id type c_ptm[1] c_ptm[2] c_ptm[3] c_centro')
df = DR.getDataframe()
data = DR.getNdArray()

atomNum = 1
atom1Data = np.array([np.array(i) for i in data[:,atomNum]])  # use this to get atom i data as shape (runs,parameters)

print(atom1Data)
#print(pd.DataFrame(data[None,:,1]))

In [ ]:
names = ['Timestep','1r','2r','3r']
a = [[[10],[1,2,3],[1,2,3],[1,2,3]],
         [[20],[1,2,3],[1,2,3],[1,2,3]],
         [[30],[1,2,3],[1,2,3],[1,2,3]],
         [[40],[1,2,3],[1,2,3],[1,2,3]]]
ar = np.array([[[10],[1,2,3],[1,2,3],[1,2,3]],
         [[20],[1,2,3],[1,2,3],[1,2,3]],
         [[30],[1,2,3],[1,2,3],[1,2,3]],
         [[40],[1,2,3],[1,2,3],[1,2,3]]])
pd.DataFrame(ar,columns = names)

ar2 = np.array([])

for row in a:
    r = np.array([])
    for elem in row:
        np.append(r,np.array(elem))
        
    np.append(ar2,r)



In [3]:
a= np.array([[1,2,3],[4,5,6],[7,8,9]])
print(a[:][0])
print(a[:,0])


[1 2 3]
[1 4 7]


# Edditable DumpReader2 class to implemented data extraction using ndarrays


In [4]:

class DumpReader2:
    def __init__(self,filename, outputLabels = ''):
        self.filename = filename
        self.outputLabels = ('ITEM: ATOMS '+ outputLabels).split()
    
    def dataExtracter(self):
        dataLines = []         # format : [[rowData1], [rowData2], [rowData3], ... ]
        self.idLabels = ['Timestep']      # format : ['Timestep', 'atomID', 'atomID', ... ]
        rowData = []                      # format : [[timestep], [x,y,z], [x,y,z], ... ]
        
        f = open(self.filename,'r')
        isFirstRun = True
        isDataLine = False          # when initially set, line in next iteration is a data line
        isTimestepLine = False      # when initially set, line in next iteration is a timestep line
        timestep = 0
        
        for line in f.readlines():  
            if line.strip() == 'ITEM: TIMESTEP' or isTimestepLine:
                if isTimestepLine:
                    isTimestepLine = False
                    timestep = int(line.split()[0])
                    rowData.append([timestep])
                    if timestep > 0:                 #assumes simulation starts at 0 timestep
                        isFirstRun = False
                else:
                    isTimestepLine = True
            
            if isDataLine:  
                data = line.split()
                if len(line.split())!= 0 and data[0].isdigit():   
                    if isFirstRun: 
                        self.idLabels.append(data[0])   # append label
                    rowData.append(line.split()[1:])   
                else:
                    isDataLine = False
                    dataLines.append(rowData)
                    rowData = []
            
            if line.split() == self.outputLabels:    
                isDataLine = True                     
                
        f.close()
        return dataLines

    #User method to get dataframe from log data
    def getDataframe(self):
        return pd.DataFrame(self.dataExtracter(),columns=self.idLabels)
    
    def getNdArray(self):
        return self.dataExtracter()
        



In [5]:
r = [[1],[1,2,3],[1,2,3]]
a = np.empty(0)
a = np.append(a,np.array([1,2,3]))
b = np.asarray(r)
for i in range(len(b)):
    b[i] = np.array(b[0])
    
print(b)

[array([1]) array([1]) array([1])]


# Function to check for convergence of data to a certain value; method of DataFileAnalyzer class


In [6]:
    import copy 
    
    def getConstValueList(self,columnLabel):
        yData = np.array(self.df[columnLabel].tolist())   # GET COLUMN DATA AS TARGET NDARRAY
        yGradient = np.gradient(yData)
        print(yGradient)
        
        
        #how to decide maxError?, minRange?
        
    def getConstValueList2(df,columnLabel,timestepInterval=10,maxError=1,minRange=50):
        #thermo data is outputedevrey timestepInterval
        
        yData = np.array(df[columnLabel].tolist()).astype(float)   # GET COLUMN DATA AS TARGET NDARRAY
        #plt.plot(yData)
        yGradient = np.gradient(yData)     # timestep is conserved
        plt.plot(yGradient[20:])
        yGGradient = np.gradient(yGradient)   # timestep is conserved
        plt.plot(yGGradient[20:])
        
        #print(yData.shape,'   ',yGradient.shape,'   ',yGGradient.shape)
        
        constValueRanges= []
        
        lowRange, uppRange = -1, -1    # -1 means var is not set
        

        for ind in range(yGGradient.shape[0]):
            #print(ind,' ',yGGradient[ind], " low: ",lowRange," upp: ",uppRange,' cond: ',(uppRange == -1) and abs(yGradient[ind]) < maxError)
            if lowRange != -1 and abs(yGGradient[ind]) < maxError:  # if lowRange is set, then can set uppRnge
                uppRange = ind
                if ind == (yGGradient.shape[0]-1):
                    constValueRanges.append([lowRange,uppRange])
            elif lowRange != -1 and uppRange != -1:
                if abs(uppRange-lowRange) > minRange:
                    #print(ind,' ',yGradient[ind], " low: ",lowRange," upp: ",uppRange)
                    constValueRanges.append([lowRange,uppRange])
                lowRange = -1
                uppRange = -1   
                
            
            if (uppRange == -1):       # if uppRange is not set then can set lowRange
                lowRange = ind
        
        print(constValueRanges)
        return constValueRanges
    
# energy = np.mean(yData[constValueRange[0],constValueRange[1]])
        
    
       
        

In [7]:
 yData = np.array(df['TotEng'].tolist()).astype(float)

# have to find a decent way to find max Error (and alittle, but its fine rn,the min Range value)
def getConstValueList3(df,columnLabel,timestepInterval=10,maxError=1,minRange=10):
        #thermo data is outputedevrey timestepInterval
        
        yData = np.array(df[columnLabel].tolist()).astype(float)   # GET COLUMN DATA AS TARGET NDARRAY
        #plt.plot(yData)
        yGradient = np.gradient(yData)     # timestep is conserved
        #plt.plot(yGradient[20:])
        yGGradient = np.gradient(yGradient)   # timestep is conserved
        #plt.plot(yGGradient[20:])
        
        #print(yData.shape,'   ',yGradient.shape,'   ',yGGradient.shape)
        
        constValueRanges= []
        
        lowRange, uppRange = -1, -1    # -1 means var is not set
        

        for ind in range(yGGradient.shape[0]):
            #print(ind,' ',yGGradient[ind], " low: ",lowRange," upp: ",uppRange,' cond: ',(uppRange == -1) and abs(yGradient[ind]) < maxError)
            if lowRange != -1 and abs(yGGradient[ind]) < maxError:  # if lowRange is set, then can set uppRnge
                uppRange = ind
                if ind == (yGGradient.shape[0]-1):
                    constValueRanges.append([lowRange,uppRange])
            elif lowRange != -1 and uppRange != -1:
                if abs(uppRange-lowRange) > minRange and np.mean(yGradient[lowRange:uppRange])<maxError:
                    #print(ind,' ',yGradient[ind], " low: ",lowRange," upp: ",uppRange)
                    constValueRanges.append([lowRange,uppRange])
                lowRange = -1
                uppRange = -1   
                
            
            if (uppRange == -1):       # if uppRange is not set then can set lowRange
                lowRange = ind
                
                
        #STD ANALYSIS
        yStd =[]
        for ind in range(yData.shape[0]):
            if ind == 0 or ind == 1:
                yStd.append(np.std([yData[ind],yData[ind+1]]))
            elif ind == yData.shape[0]-1 or ind == yData.shape[0]-2:
                yStd.append(np.std([yData[ind],yData[ind-1]]))
            else:
                yStd.append(np.std([yData[ind-2],yData[ind-1],yData[ind],yData[ind+1],yData[ind-2]]))
        plt.plot(yStd[10:])
            
            
        
        print(constValueRanges)
        
        
        
        return constValueRanges

NameError: name 'df' is not defined

In [8]:
lst = getConstValueList3(df,'TotEng')

'''
yData = np.array(df['TotEng'].tolist()).astype(float)[10:]

subData = []
for rang in lst:
    subData.append(yData[rang[0]:rang[1]+1])
    
for subD in subData:
    abc = getConstValueList2(pd.DataFrame(subD,columns=['ySubData']),'ySubData')
    #dftest = pd.DataFrame(subD,columns=['ySubData'])
    #print(dftest['ySubData'])
    print()
'''

NameError: name 'getConstValueList3' is not defined

In [9]:
yData = np.array(df['TotEng'].tolist()).astype(float)[10:]   # GET COLUMN DATA AS TARGET NDARRAY
plt.plot(yData)

#lst.append([11,146])

for rang in lst:
    y=np.ones(rang[1]-rang[0]+1)*np.mean(yData[rang[0]:rang[1]])
    plt.plot(np.arange(rang[0],rang[1]+1),y)
    #print(y)


NameError: name 'df' is not defined

In [10]:
#Y is vector of data

dataBin = 1 #ORANGE
binDivisibleY = yData[yData.shape[0]%dataBin:]

binnedY = binDivisibleY[None,:].reshape(dataBin,int(binDivisibleY.shape[0]/dataBin)) #binned data is along column
meanBinnedY = np.mean(binnedY,axis=0)

stdBin = 2 #GREENE
binDivisibleMeanY = meanBinnedY[meanBinnedY.shape[0]%stdBin:]
binnedMeanY = binDivisibleMeanY[None,:].reshape(stdBin,int(binDivisibleMeanY.shape[0]/stdBin)) #binned data is along column
stdBinnedMeanY = np.std(binnedMeanY,axis=0)

print(yData.shape,binDivisibleY.shape,binnedY.shape,meanBinnedY.shape, binDivisibleMeanY.shape,binnedMeanY.shape,stdBinnedMeanY.shape)

#plt.plot(yData)
plt.plot(meanBinnedY)
plt.plot(stdBinnedMeanY)
              



NameError: name 'yData' is not defined

# BETTER READER CLASS FOR DUMP AND LOG OUTPUTS

In [11]:
'''
log files output global quantities
dump files output peratom quatities

assumes run start at 0 timestep,
assumes first line is not a dataline
#can edit class to take in only filename and assume all log filenames begin with log.<filename>, so seperate filename by '.' and if first string is log then set output type as 'log'
for dump:
    assumes id is first row  # can fix this by looping through labels list and finding index and checking that index for atom ids
    assumes ids are in same row order for in each timestep


'''

class OutputReader:
    def __init__(self,filename,outputType,dataLabels,sepUniqueRun=False):
        self.filename = filename
        self.outputType = outputType
        self.datafile = None   # singe datafile for log/global qauntities or list of datafiles where each elem is a df for the ith atom
        self.ndArray = None # singe ndarray for log/global qauntities or ndarray of ndarrays where each elem is a nda for the ith atom
        self.dataLabels = dataLabels.split()
        self.sepUniqueRun = sepUniqueRun
    
    def getDataFrame(self):
        if self.datafile is None:
            if self.outputType == 'log':
                if self.sepUniqueRun == False:
                    self.datafile = pd.DataFrame(self.getNDArray(),columns=self.dataLabels)
                else:
                    runKeys = []
                    runDF = []
                    for ind in range(self.getNDArray().shape[0]):
                        runDF.append(pd.DataFrame(self.getNDArray()[ind],columns=self.dataLabels))
                        runKeys.append(ind)
                    self.datafile = pd.concat(runDF,keys=runKeys,names=['Run Number','Row Number'])
            elif self.outputType == 'dump':
                print('implement this code')
                return
            else:
                print('Error: cannot read output type '+str(self.outputType)) # should raise an error
                return
        
        return self.datafile
    
    def getNDArray(self):
        if self.ndArray is None:
            self.ndArray = np.array(self.extractData())
        return self.ndArray
            
    def getDataLabels(self):
        return self.dataLabels
        
    def editReader(self,newFile=None,newType=None,newDataLabels=None,sepUniqueRun=None):  # relies on reader to set correct parameters      
        readerChanged = False          # If any reader parameter changed then have to re-ExtractData
        
        if newFile is not None:
            self.filename = newFile
            readerChanged = True
        if newType is not None:
            self.outputType = newType 
            readerChanged = True
        if newDataLabels is not None:
            self.dataLabels = newDataLabels
            readerChanged = True
        if sepUniqueRun is not None:
            self.sepUniqueRun = sepUniqueRun
            readerChanged = True
            
        if readerChanged:     
            self.datafile = None 
            self.ndArray = None
            
        return
    
    def extractData(self):
        didNotfindDataLabels = True  # check if data labels were found atleast once
        
        if self.outputType == 'dump':
            dataHeader = 'ITEM: ATOMS'.split() + self.dataLabels 
        elif self.outputType == 'log':  
            dataHeader =  self.dataLabels
        
        f = open(self.filename,'r')
        runData = []
        if self.sepUniqueRun:            
            dataCollection = []
        isDataLine = False                 # identify if current line is file data line

        for line in f.readlines():          # for first iteration, assumes first line will not be a data line
            if isDataLine:                  #true when line is data line and adds it to data collection
                if len(line.split()) != 0 and line.split()[0].isdigit():     # assumes data line starts an numeric value
                    runData.append(line.split())
                else:
                    isDataLine = False
            
            if line.split() == dataHeader:        #find the output label line preceding the rows of data
                didNotfindDataLabels = False 
                isDataLine = True                       # sets following line as a file dataline
                if self.sepUniqueRun:             # make data lines array of arrays for each run
                    dataCollection.append(np.array(runData))
                    runData = []
        f.close()
        
        if didNotfindDataLabels:
            print('ERROR: did not find data with current datalabels')
            return []   # should raise an error instead of returning empty list
        else:
            if not self.sepUniqueRun:
                return runData
            else:
                dataCollection.append(np.array(runData))
                return dataCollection[1:] # first item is always an empty list if data is found

        
        
        

same files as above but version 2 trying to make reader work with dump files

In [53]:
'''
log files output global quantities
dump files output peratom quatities

assumes run start at 0 timestep,
assumes first line is not a dataline
#can edit class to take in only filename and assume all log filenames begin with log.<filename>, so seperate filename by '.' and if first string is log then set output type as 'log'

for dump:
    assumes id is first row  # can fix this by looping through labels list and finding index and checking that index for atom ids
    assumes ids are in same row order for in each timestep
    assumes no clear way in datafile to differentiate run commans, in log data is grouped by run commands

'''

class OutputReader:
    def __init__(self,filename,outputType,dataLabels,sepUniqueRun=False):
        self.filename = filename
        self.outputType = outputType
        self.datafile = None   # singe datafile for log/global qauntities or list of datafiles where each elem is a df for the ith atom
        self.ndArray = None # singe ndarray for log/global qauntities or ndarray of ndarrays where each elem is a nda for the ith atom
        self.dataLabels = dataLabels.split()
        self.sepUniqueRun = sepUniqueRun
    
    def getDataFrame(self):
        if self.datafile is None:
            if self.outputType == 'log':
                if self.sepUniqueRun == False:
                    self.datafile = pd.DataFrame(self.getNDArray(),columns=self.dataLabels)
                else:
                    runKeys = []
                    runDF = []
                    for ind in range(self.getNDArray().shape[0]):
                        runDF.append(pd.DataFrame(self.getNDArray()[ind],columns=self.dataLabels))
                        runKeys.append(ind)
                    self.datafile = pd.concat(runDF,keys=runKeys,names=['Run Number','Row Number'])
            elif self.outputType == 'dump':
                runKeys, runDF = [], []
                for ind in range(self.getNDArray().shape[0]):
                    runDF.append(pd.DataFrame(self.getNDArray()[ind],columns=['step']+self.dataLabels))
                    runKeys.append(ind+1)
                self.datafile = pd.concat(runDF,keys=runKeys,names=['Atom Number','Row Number'])
                
        return self.datafile
    
    def getNDArray(self):
        if self.ndArray is None:
            if self.outputType == 'log':
                self.ndArray = np.array(self.extractLogData())
            elif self.outputType == 'dump':
                self.ndArray = np.array(self.extractDumpData())
            else:
                print('Error: cannot read output type '+str(self.outputType)) # should raise an error
                return
        return self.ndArray
            
    def getDataLabels(self):
        return self.dataLabels
        
    def editReader(self,newFile=None,newType=None,newDataLabels=None,sepUniqueRun=None):  # relies on reader to set correct parameters      
        readerChanged = False          # If any reader parameter changed then have to re-ExtractData
        
        if newFile is not None:
            self.filename = newFile
            readerChanged = True
        if newType is not None:
            self.outputType = newType 
            readerChanged = True
        if newDataLabels is not None:
            self.dataLabels = newDataLabels
            readerChanged = True
        if sepUniqueRun is not None:
            self.sepUniqueRun = sepUniqueRun
            readerChanged = True
            
        if readerChanged:     
            self.datafile = None 
            self.ndArray = None
            
        return
    
    def extractLogData(self):
        didNotfindDataLabels = True  # check if data labels were found atleast once
        
        dataHeader =  self.dataLabels
        if self.sepUniqueRun:            
            dataCollection = []

        f = open(self.filename,'r')
        runData = []
        isDataLine = False                 # identify if current line is file data line

        for line in f.readlines():          # for first iteration, assumes first line will not be a data line
            if isDataLine:                  #true when line is data line and adds it to data collection
                if len(line.split()) != 0 and line.split()[0].isdigit():     # assumes data line starts an numeric value
                    runData.append(line.split())
                else:
                    isDataLine = False
            
            if line.split() == dataHeader:        #find the output label line preceding the rows of data
                didNotfindDataLabels = False 
                isDataLine = True                       # sets following line as a file dataline
                if self.sepUniqueRun:             # make data lines array of arrays for each run
                    dataCollection.append(np.array(runData))
                    runData = []
        f.close()
        
        if didNotfindDataLabels:
            print('ERROR: did not find data with current datalabels')
            return []   # should raise an error instead of returning empty list
        else:
            if not self.sepUniqueRun:
                return runData
            else:
                dataCollection.append(np.array(runData))
                return dataCollection[1:] # first item is always an empty list if data is found

    def extractDumpData(self):
        dataHeader = 'ITEM: ATOMS'.split() + self.dataLabels 

        f = open(self.filename,'r')
        datalines = [line.strip().split() for line in f.readlines()]
        numAtoms = int(datalines[datalines.index('ITEM: NUMBER OF ATOMS'.split())+1][0])
        widthOfHeader = datalines.index('ITEM: ATOMS'.split()+self.dataLabels)+1 
        totData = []
        numRows = int(len(datalines)/(numAtoms+widthOfHeader))
        linesBeforeAtomRepeat = numAtoms+widthOfHeader
        
        for i in range(numAtoms):
            atomDataLineInd = widthOfHeader+i
            timestepArray = [datalines[datalines.index('ITEM: TIMESTEP'.split())+1+(ind*linesBeforeAtomRepeat)] for ind in range(numRows)]
            totData.append(np.array([timestepArray[ind]+datalines[atomDataLineInd+(ind*linesBeforeAtomRepeat)] for ind in range(numRows)]))
 
        f.close()
       
        return totData


In [54]:
a = OutputReader('log.interfaceTracking','log','Step Atoms Temp Press TotEng Enthalpy')

#print(a.getNDArray())
#print(a.getDataFrame())
#print(a.getDataLabels())
a.editReader(sepUniqueRun=True)
#print(a.filename,' ',a.outputType,' ',a.dataLabels,' ',a.sepUniqueRun,' ',a.datafile,' ',a.ndArray)
#print(a.getDataFrame())
#print(a.getNDArray())

In [84]:
'''
nd array looks like:
[id_1_data,id_2,data,...]
[[step,param1,param2,...],[step,param1,param2,...],[step,param1,param2,...],[step,param1,param2,...]]

if

'''

a = OutputReader('pos.XYZ','dump','type id x y z c_ptm[1] c_centro c_cna')

print(a.dataLabels,'\n')
print(a.getDataFrame().xs(100))
#print(a.getNDArray()[100])

['type', 'id', 'x', 'y', 'z', 'c_ptm[1]', 'c_centro', 'c_cna'] 

            step type   id        x        y        z c_ptm[1]     c_centro  \
Row Number                                                                    
0              0    1  100     7.26    9.075    1.815        1  5.04871e-29   
1             10    1  100     7.26    9.075    1.815        1  5.04871e-29   
2             20    1  100  7.26048  9.07628  1.81562        1    0.0107859   
3             30    1  100  7.26076  9.08028  1.81758        1    0.0563253   
4             40    1  100  7.25965   9.0881  1.82121        1     0.116235   
5             50    1  100   7.2564  9.09992  1.82636        1     0.142991   
6             60    1  100  7.25187  9.11468  1.83255        1     0.155105   
7             70    1  100  7.24786   9.1306  1.83914        1     0.238659   
8             80    1  100  7.24451  9.14553  1.84558        1     0.342857   
9             90    1  100   7.2408  9.15639  1.85113        1    

In [ ]:
one = pd.DataFrame(np.array([['a','a','a'],['b','b','b'],['c','c','c']]),columns=['let1','let2','let3'])
two = pd.DataFrame(np.array([['d','d','d'],['f','f','f'],['g','g','g']]),columns=['let1','let2','let3'])

df = pd.concat([one,two],keys=['1','2'])
print(df)

In [166]:
ar1 = np.array(['1x1','1y1','1z1','1w1'])
ar2 = np.array(['2x1','2y1','1z1','1w1'])
ar3 = np.array(['3x1','3y1','3z1','3w1'])


ar11 = np.array(['1x2','1y2','1z2','1w2'])
ar22 = np.array(['2x2','2y2','2z2','2w2'])
ar33 = np.array(['3x2','3y2','3z2','3w2'])

timesteps = 2
numAtoms = 3
fin1 = np.array([ar1,ar2,ar3])[:,None,:]

fin2 = np.array([ar11,ar22,ar33])[:,None,:]

print(np.append(fin1,fin2,axis=1))

[[['1x1' '1y1' '1z1' '1w1']
  ['1x2' '1y2' '1z2' '1w2']]

 [['2x1' '2y1' '1z1' '1w1']
  ['2x2' '2y2' '2z2' '2w2']]

 [['3x1' '3y1' '3z1' '3w1']
  ['3x2' '3y2' '3z2' '3w2']]]


In [243]:
# New Method

f = open('atomParameters.data','r')
datalines = [line.strip() for line in f.readlines()]

numAtoms = 648
widthOfHeader = 9
totData = []
numRows = int(len(datalines)/(numAtoms+widthOfHeader))
print(numRows)
lineBeforeAtomRepeat = numAtoms+widthOfHeader
for i in range(numAtoms):
    atomDataLineInd = widthOfHeader+i
    totData.append(np.array([datalines[atomDataLineInd+(ind*lineBeforeAtomRepeat)] for ind in range(numRows)]))
    
print(np.array(totData)[0])


102
['1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653'
 '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653'
 '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653'
 '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653'
 '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653'
 '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653'
 '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653'
 '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653'
 '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653'
 '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653'
 '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653'
 '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653'
 '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653'
 '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653'
 '1 2 0 inf 0 19.7653' '1 2 0 inf 0 19.7653' '1 2 0 inf 0 

In [29]:
myArr = np.array([np.array([np.array(['1x1','1y1','1z1']),np.array(['1x2','1y2','1z2'])]),[['2x1','2y1','2z1'],['2x2','2y2','2z2']],[['3x1','3y1','3z1'],['3x2','3y2','3z2']]])
print(myArr)

#(numAtoms,numTimesteps,numParam)

[[['1x1' '1y1' '1z1']
  ['1x2' '1y2' '1z2']]

 [['2x1' '2y1' '2z1']
  ['2x2' '2y2' '2z2']]

 [['3x1' '3y1' '3z1']
  ['3x2' '3y2' '3z2']]]


In [30]:
print([data[9],data[9+1*lineBeforeAtomRepeat]])

IndexError: index 666 is out of bounds for axis 0 with size 101

In [229]:
print(np.array([2])+np.array([1]))

[3]
